<div style="text-align:right;color:blue">version id: __VERSION_ID__</div>

# Object Oriented Programming (OOP) - Part I: User defined types

<center><i>Programs must be written for people to read, and only incidentally for machines to execute.</i></center>
<p style="text-align:center;">(Abelson and Sussman, 1985)</p>

In the first part of this course we mentioned that "*programming is about writing instructions (=computer code) that a computer can execute*". Our first consideration when writing code therefore is *correctness*, i.e. that the code should do what we expect it to do. Often the next consideration is *efficiency* or *speed*. However, when it comes to writing larger pieces of code that achieve more complex computational tasks or that aim to survive more than a few weeks' use by the programmer her(him)self, we need to consider aspects beyond correctness and speed. 

In this notebook we will learn about Object Oriented Programming (OOP) in Python, which consists of some programming concepts and practices that are extremely helpful in making complex programs easier to write, and leads to readable, reusable and maintainable programs. It is no surprise that OOP becomes absolutely essential when writing large scale programs and is used in virtually every piece of software that you use on a daily basis. As you will see from the examples we will cover, however,  OOP is just as helpful for writing good programs for mathematical purposes! 

<hr style="height: 2px">

### What you will learn
In this notebook we will cover the following topics:

* User defined types (classes) 
* Creating objects or instances of user defined types
* Attributes (or properties) of objects
* Functions that act on objects
* Attributes of a class

<hr style="height: 2px">

*&#169; Pranav Singh, University of Bath 2021-2025. This problem sheet is copyright of Pranav Singh, University of Bath. It is provided exclusively for educational purposes at the University and is to be downloaded or copied for your private study only. Further distribution, e.g. by upload to external repositories, is prohibited.*

## Linear functions

Consider degree 1 polynomials in $x \in \mathbb{R}$. These are linear functions $f(x) = ax + b$ with integer coefficients $a,b \in \mathbb{Z}$, and can be represented by a list of integers `[a,b]`. For instance, we store $f(x) = 3x-4$ as `[3,-4]`.

In [ ]:
f = [3,-4]

Just like Tickables 10, we can define a function that helps us evaluate a linear function (degree 1 polynomial) at a point $x\in \mathbb{R}$. 

In [ ]:
def evaluate(g, x):
    a = g[0]
    b = g[1]
    return a*x + b

print(evaluate(f, 3/7))
print(evaluate(f,0))
print(evaluate(f,-1))

Note that even though the coefficients $a$ and $b$ are integers and the point $x=3/7$ is rational, $f(x)$ is not computed as a rational. This is because $3/7$ is stored in the computer as a floating point number. In order to get the answer in terms of rational numbers, $3 \times \frac{3}{7} - 4 = \frac{9-28}{7} = -\frac{19}{7}$, we would have to use our implementation of rational numbers from Lecture 14. This implementation is available in the module `rational.py` in the folder `week04` and we can utilise it by importing it using `import rational`.

In [ ]:
import rational

def evaluate_rat(g, x):
    a = g[0]
    b = g[1]
    return rational.add_int(b, rational.mul_int(a, x))

x = [3, 7]
f = [3, -4]

print(rational.to_str(evaluate_rat(f, x)))

<br>

#### Issues with this approach

Let us pause to make a few observations. Since `a` and `b` are integers (not rationals), in 

```Python
return rational.add_int(b, rational.mul_int(a, x))
```

we have to be very careful to use `add_int` and `mul_int` instead of `add` and `mul`. In mathematics, we could simply write $3 \times \frac{3}{7}$ and $\frac{9}{7} - 4$, knowing that $3$ can always be interpreted as $\frac{3}{1}$ and $-4$ as $\frac{-4}{1}$. However, using `rational.mul` gives us an error.


In [ ]:
rational.mul(3,[3,7])

<br> 

We have to be careful of the **order of the parameters** for `add_int` and `mul_int`. For instance `rational.mul_int([3,7],3)` gives an error. In mathematics, we don't have to care about the order of multiplication of rationals and integers, $3 \times \frac{3}{7} = \frac{3}{7} \times 3 = \frac{9}{7}$.

In [ ]:
rational.mul_int([3,7],3)

<br>

Nothing stops us from treating `f` which is a linear function (degree 1 polynomial), as a rational number

In [ ]:
print(rational.to_str(f))

or from treating `x` (which is a rational number) as a linear function

In [ ]:
print(evaluate(x,3))

or, both, leading to potentially ambiguous situation such as in the case of `evaluate_rat` where we have to think carefully to recall whether the first parameter is a linear function and the second a rational number, or vice-versa.

In [ ]:
print(evaluate_rat([3,4],[1,2]))

The code quickly becomes **unreadable**. For instance, ask yourself which of the following is more readable:

```Python
    rational.add_int(b, rational.mul_int(a, x))
```

&emsp;&emsp;or 

``` Python
    a*x + b 
```

There are some deep issues here. For instance


* When seen without context, what does `s = [5,7]` represent? Is it the rational number $s = \frac{5}{7}$ or the linear function $s(x) = 5x+7$?

* Which addition operation should be used on `s = [5,7]` and `r = [-1,2]`? Should they be added as rational numbers or as linear functions? 

## User defined types

While it is possible to represent rational numbers, linear functions, lines in 2D, points in 2D space etc. using inbuilt python datatypes such as lists, this can create a lot of ambiguity. Not only does it make the code much harder for someone else to read and understand, but it can also be a recipe for a code full of unpredictable behaviours, and errors that result from accidentally using the wrong function or the wrong order of parameters. We have also seen that notation such as `rational.add_int` is significantly more cumbersome than simply being able to use operators like `+`. 

In this section we will see how we can define our own datatypes in Python. In Python a type is also called a `class`. We can create a user defined type `LinearFunction` using the `class` syntax.

In [ ]:
class LinearFunction(object):
    '''Linear functions on R'''

The header `class LinearFunction(object)` indicates that `LinearFunction` is a `class` and that each linear function is a kind of `object`. The second line is a *docstring* which should provide a very brief description of our new class in a human readable format. As usual, including a docstring is not essential but it is a very good practice. 

We can find information about the class `LinearFunction` using the `?` syntax:

In [ ]:
?LinearFunction

The above information tells us that `LinearFunction` is a `type`. So we have successfully created our first user defined type in Python!

We can now create a linear function $f$ which will be an *object* of *type* `LinearFunction`. The **Init signature** tells us how to do so. Specifically, we need to call `LinearFunction()` -- i.e. we call the name of our new class as though it is a function. Calling the class name as a function creates a new object of the type defined by the class.

In [ ]:
f = LinearFunction()
print(type(f))

We can also use `?` syntax on the object `f` to get information on it:

In [ ]:
?f

## Objects (or Instances)

The new object $f$ is called an *instance* of the class `LinearFunction`, and the act of its creation is called an *instantiation* of `LinearFunction`. We can use the `isinstance(x,cls)` function to check whether an object 'x' is an instance of the class 'cls'.

In [ ]:
isinstance(f, LinearFunction)

The use of `isinstance` is not limited to user defined class. Indeed, it can be used for all datatype. For instance, we can also check whether a variable is of built in datatype such as `int`, `str`, `float` etc.

In [ ]:
s = 'hello'
print(isinstance(s, str))
print(isinstance(s, int))

The number $4$, as you would recall, can be stored either as an `int` or a `float` depending on whether we write it as `4` or `4.0`, respectively.

In [ ]:
xi = 4
print(isinstance(xi, int))
print(isinstance(xi, float))

In [ ]:
xf = 4.0
print(isinstance(xf, int))
print(isinstance(xf, float))

We can also check whether an object is an instance of one of the classes among multiple classes `cls1, cls2, ..., clsn` by using `isinstance(x, (cls1, cls2, ..., clsn))`. For example, to check if `xi` and `xf` are numbers, we may want to check whether they are either `int` or `float`. 

In [ ]:
print(isinstance(xi, (int, float)))
print(isinstance(xf, (int, float)))

As we have seen, the variables `f`, `s`, `xi` and `xf` store objects of type `LinearFunction`, `str`, `int` and `float`, respectively. The common aspect is that each of them is an *object* (of some type or the other). For instance, `f` is an *object* of type (or class) `LinearFunction`, `s` is an *object* of type (or class) `str`, etc. Python defines a universal class called `object` and any instance of any class automatically is also an instance of the class `object`. Thus, `f`, `s`, `xi` and `xf` are also instances of the class `object`.

In [ ]:
print(isinstance(f, object))
print(isinstance(s, object))
print(isinstance(xi, object))
print(isinstance(xf, object))

Recall how we defined the class `LinearFunction` using `class LinearFunction(object)`. The use of `object` in the definition makes every instance of `LinearFunction` also an instance of the universal class `object`. Note that `f` is an instance of type `LinearFunction` as well as of type `object`. Thus something can be an instance of more than one class. We will study this in more detail when we come across the concept of *inheritance*.

## Attributes of an object

Currently `f` has no data. For it to represent a linear function $f(x) = ax + b$, we should also specify the coefficients $a$ and $b$. Let us say we wish to create the linear function $f(x) = 5x +7$. We should store the values $5$ and $7$ in the *attributes* `a` and `b`, respectively. 

In [ ]:
f.a = 5
f.b = 7

The *attributes* `a` and `b` are associated with the *object* `f` and are accessed using the *dot* notation. Attributes can be seen as *properties* of an object.

In [ ]:
f.a

This notation tells us to go inside the object `f` and find the value of `b` stored in there. There should be no confusion with a *variable* of the same name. For instance, `f.b` is defined, but we have not yet defined the variable `b` in this notebook. 

In [ ]:
b

Even if the variable `b` was defined, it does not refer to `f.b`, which is the value of `b` stored *inside* the object `f`.

In [ ]:
b = -1
print(b)
print(f.b)

## Functions

Since we can access the attributes of an instance of class `LinearFunction`, we can create a `to_str_lf` function which creates a user readable string representation of a linear function. For instance, `f = [5,7]` can be represented by the string `'5x + 7'`.

In [ ]:
def to_str_lf(g):
    '''Returns a string representation of a LinearFunction'''
    return str(g.a) + "x + " + str(g.b);

In [ ]:
to_str_lf(f)

Here we have used the postfix `lf` to indicate that this function acts on instances of `LinearFunctions`.

Similarly, we can implement a function to evaluate a linear function `g` at a given point `x`.

In [ ]:
def evaluate_lf(g, x):
    '''Evaluates the linear function g at the point x and returns g(x)'''
    return g.a*x + g.b

In [ ]:
print(evaluate_lf(f, 3))

The postfix `lf` distinguishes this function from our initial implementation of `evaluate` function, whose first parameter needs to be a list of two numbers, unlike `evaluate_lf` where the first parameter `g` must be a `LinearFunction`. 

Using `to_str_lf` and `evaluate_lf`, we can create more readable and informative output:

In [ ]:
print('The value of '+to_str_lf(f)+ ' at x = 3 is ' + str(evaluate_lf(f,3)))

<br>

We can also create a function to add two linear functions $f$ and $g$, which should return another `LinearFunction` $r$ such that 

$$
r.a = f.a + g.a \texttt{ and }  r.b = f.b + g.b
$$

In [ ]:
def add_lf(f, g):
    '''Returns the sum of two LinearFunctions'''
    r = LinearFunction()
    r.a = f.a + g.a
    r.b = f.b + g.b
    return r

Note that we have to create `r` as a new *instance* of `LinearFunction` using `r = LinearFunction()` before setting its attributes `a` and `b`.

In [ ]:
f1 = LinearFunction()
f1.a = 5
f1.b = 7

f2 = LinearFunction()
f2.a = 3
f2.b = -4

f3 = add_lf(f1, f2)
print('The sum of ' + to_str_lf(f1) + ' and ' + to_str_lf(f2) + ' is ' + to_str_lf(f3))

<br> 

We can also create a function `random_lf` that outputs a random linear function. To do this we need the `random` function `np.random.random()` which returns a random number between $0$ and $1$. Let's print 3 different random numbers using this function:

In [ ]:
import numpy as np
print(np.random.random())
print(np.random.random())
print(np.random.random())

We use this in the function `random_lf` which returns a random linear function:

In [ ]:
import numpy as np

def random_lf():
    '''Returns a random LinearFunction'''
    g = LinearFunction()
    g.a = np.random.random()
    g.b = np.random.random()
    return g

In [ ]:
f = random_lf()
print(to_str_lf(f))

* The function `evaluate_lf` acts on an instance of `LinearFunction` (`g`) and a `float` (`x`) and produces a `float` (`g(x)`).

* The function `to_str_lf` acts on an instance of `LinearFunction` and produces a `str`.

* The function `add_lf` acts on two instances of `LinearFunction` (`f` and `g`) and returns another instance of `LinearFunction`, `r` (`r=f+g`). 

* The function `random_lf` takes no input, and returns an instance of `LinearFunction`. 

In mathematical notation, we say that these functions are maps:

* `evaluate_lf` : `LinearFunction` $\times$ `float` $\rightarrow$ `float`.

* `to_str_lf` : `LinearFunction` $\rightarrow$ `str`.

* `add_lf` : `LinearFunction` $\times$ `LinearFunction` $\rightarrow$ `LinearFunction`.

* `random_lf` : `null` $\rightarrow$ `LinearFunction`.


Thus, functions can act on instances of `LinearFunction` and can also return instances of `LinearFunction`.

## Missing attributes

While we can create a string representation for the linear function `f1`,

In [ ]:
to_str_lf(f1)

the following attempt at creating a string representation for `f4` fails.

In [ ]:
f4 = LinearFunction()
to_str_lf(f4)

This is because we have neglected to define the attributes `a` and `b` for the instance `f4`. We can explicitly check whether the instance has an attribute called `a` using the `hasattr` function:

In [ ]:
hasattr(f4, 'a')

In [ ]:
hasattr(f1, 'a')

In the next lecture we will see how to avoid such a possibility - i.e. we would like to forbid the creation of an object of type `LinearFunction` unless the values of both parameters `a` and `b` are specified.

For now, you should exercise great care and ensure that you (or your functions such as `add_lf`) appropriately define the values of the attributes `a` and `b` for every instance of `LinearFunction`.

## Check your understanding

The solutions to these excercises are provided at the very end of this notebook.

**Q1)** Which of the following is the correct way to create an instance of `LinearFunction` with the name `f`?

a.
```Python
LinearFunction(f)
```

b.
```Python
f = LinearFunction()
```

c.
```Python
f = LinearFunction
```

**Q2)** Let's say that `f` is an instance of `LinearFunction`. What is the value of the attribute `a` of `f` after the following code:

```Python
f.a = 4
a = 7
```

**Q3)** Let's say that `f` and `g` are instances of `LinearFunction`. What does the following code print?

```Python
f.a = 1
f.b = -3
g.a = 2
g.b = 5
add_lf(f,g)
print(to_str_ln(f))
```


<br>
<br>

# Advanced (Optional): Attributes of a class

In Python, classes can also have attributes. For instance, we can create an attribute `points` of the class `LinearFunction` with the value `[0,0.5,1,1.5]` by adding

```Python
points = [0, 0.5, 1, 1.5]
```

in the definition of the class `LinearFunction`.

In [ ]:
class LinearFunction(object):
    """Linear functions on R"""
    
    points = [0.0, 0.5, 1.0, 1.5]

We can access this attribute of the class `LinearFunction` using the dot notation on the class name `LinearFunction`.

In [ ]:
LinearFunction.points

A class attribute may be useful if it has to be used for many instances. For instance, say we have an interest in computing the average of the linear functions at the points stored in `LinearFunction.points`. We can create a function `sample_average_lf` which does this. 

In [ ]:
evaluate_lf(f1,4)

In [ ]:
LinearFunction.points

In [ ]:
def sample_average_lf(g):
    '''This function computes the average value of the function g at the points LinearFunction.points'''
    n = len(LinearFunction.points)
    sum = 0                                                # accumulator for computing sum
    for i in range(n):
        value = evaluate_lf(g, LinearFunction.points[i])   # value of the function g at the ith point
        sum = sum + value
        
    average = sum/n
    return average

In [ ]:
sample_average_lf(f1)

In [ ]:
sample_average_lf(f2)

### Some issues with class attributes

A *class attribute* is also available to *instances* of the class `LinearFunction` using the dot notation on the instance.

In [ ]:
f4 = LinearFunction()
f4.points

Unlike *object attributes* `a` and `b`, which we defined earlier, the *class attribute* `points` is shared across all instances of the class `LinearFunction`. For instance, see what happens when we change the value of `LinearFunction.points`. The value of the *class attribute* `points` is changed for the newly created instance `f5` but also for the old instance `f4`.

In [ ]:
LinearFunction.points = [0,5,10]
f5 = LinearFunction()

print(LinearFunction.points)
print(f4.points)
print(f5.points)

**Warning:** If we assign a new value to `f5.points`, this creates a new *object attribute* with the name `points` within `f5`. 

In [ ]:
f5.points = [-1, 0, 1]

print(f5.points)
print(f4.points)
print(LinearFunction.points)

Now `f5.points` is an *object attribute* and we cannot use it to refer to the *class attribute* `points`! However, we did not set the value of `f4.points`, and it still refers to the *class attribute* `points`.

This behaviour can obviously be very confusing, and **safe practices** to avoid such ambiguity are:

* Use class attributes sparingly, and only if necessary.

* If possible, avoid changing the value of class attribute.

* Even though class attribute is shared across all instances, refer to the attribute using the notation *classname*.*attribute* and avoid *instance*.*attribute*. 

* If you must change the value of the class attribute, it is particularly crucial that you do so by using *classname*.*attribute* = *value* (**not** *instance*.*attribute* = *value*). 

* Never use the same name for a class attribute and an object attribute.


## Solutions to "Check your understanding"

**Q1)** Answer: b.

**Q2)** Answer: 4.

**Q3)** Answer: `'1x - 3'`. Note that value of `f` does not change due to the `add_lf` call.